In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("EXCHUS.csv")
tsValues = data['EXCHUS']
import math
tsValuesLog = [math.log(ele) for ele in tsValues]

In [2]:
PredictionOutputMAE = pd.DataFrame(index=['monthly','3_month','6_month','12_month'])
PredictionOutputRMSE = pd.DataFrame(index=['monthly','3_month','6_month','12_month'])
startPrediction = 100
totalData = len(tsValuesLog)

In [3]:
totalData

461

In [4]:
startPrediction=300

In [5]:
def mean_error(pred,real):
    return np.mean([pred[i]-real[i] for i in range(len(pred))])
def mean_absolute_error(pred,real):
    return np.mean([np.abs(pred[i]-real[i]) for i in range(len(pred))])
def root_mean_square_error(pred,real):
    return np.sqrt(np.mean([pow(pred[i]-real[i],2) for i in range(len(pred))]))

In [12]:
PredictionOutputRMSE

,AR_1_1,RandomWalk,AR_Diff_Mean
monthly,0.007134,0.007606,0.008292
3_month,0.017257,0.018230,0.021992
6_month,0.027466,0.029523,0.039300
12_month,0.041231,0.045839,0.070358


In [10]:
from statsmodels.tsa.arima_model import ARMA
mae_ = []
rmse_ = []
test_queue = [[1,1],[0,1],[1,0]]
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        single_pred = []
        for [p,q] in test_queue:
            try:
                model = ARMA(tsValuesLog_d_1[i:i+startPrediction],order=(p,q))
                result = model.fit()
                single_pred.append(sum(result.predict(start=startPrediction,end=startPrediction+l-1))+tsValuesLog[i+startPrediction-1])
            except ValueError:
                single_pred.append(tsValuesLog[i+startPrediction-1])
        pred.append(np.mean(single_pred))
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_Diff_Mean'] = mae_
PredictionOutputRMSE['AR_Diff_Mean'] = rmse_

C:\Users\v-weiyzh\AppData\Roaming\Python\Python36\site-packages\scipy\signal\signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
C:\Users\v-weiyzh\AppData\Roaming\Python\Python36\site-packages\scipy\signal\signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]
C:\Users\v-weiyzh\AppData\Roaming\Python\Python36\site-packages\scipy\signal\signaltools.py:1350: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead o

In [8]:
mae_ = []
rmse_ = []
p=1
q=1
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        model = ARMA(tsValuesLog[i:i+startPrediction],order=(p,q))
        result = model.fit()
        pred.append(result.predict(start=startPrediction,end=startPrediction+l-1)[l-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_'+str(p)+'_'+str(q)] = mae_
PredictionOutputRMSE['AR_'+str(p)+'_'+str(q)] = rmse_

In [9]:
#benchmark
mae_ = []
rmse_ = []
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        pred.append(tsValuesLog[startPrediction-1+i])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
PredictionOutputMAE['RandomWalk'] = mae_
PredictionOutputRMSE['RandomWalk'] = rmse_

In [7]:
PredictionOutputMAE

,RandomWalk
monthly,0.005169
3_month,0.013316
6_month,0.022940
12_month,0.037548


In [8]:
#test performance random walk with drift
mae_ = []
rmse_ = []
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        monthly_change = [tsValuesLog[j]-tsValuesLog[j-l] for j in range(i+l,i+startPrediction)]
        mean_monthly_change = np.mean(monthly_change)
        pred.append(tsValuesLog[startPrediction+i-1]+mean_monthly_change)
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['RandomWalkWithDrift'] = mae_
PredictionOutputRMSE['RandomWalkWithDrift'] = rmse_

In [9]:
PredictionOutputMAE

,RandomWalk,RandomWalkWithDrift
monthly,0.005169,0.006898
3_month,0.013316,0.018713
6_month,0.022940,0.033806
12_month,0.037548,0.060703


In [10]:
%%time
from statsmodels.tsa.arima_model import ARMA
#out-of-sample-test
mae_ = []
rmse_ = []
p=1
q=1
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        try:
            model = ARMA(tsValuesLog[i:i+startPrediction],order=(p,q))
            result = model.fit()
            pred.append(result.predict(start=startPrediction,end=startPrediction+l-1)[l-1])
        except ValueError:
            pred.append(tsValuesLog[startPrediction+i-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_'+str(p)+'_'+str(q)] = mae_
PredictionOutputRMSE['AR_'+str(p)+'_'+str(q)] = rmse_

Wall time: 1min 20s


In [11]:
%%time
mae_ = []
rmse_ = []
p=2
q=0
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        try:
            model = ARMA(tsValuesLog[i:i+startPrediction],order=(p,q))
            result = model.fit()
            pred.append(result.predict(start=startPrediction,end=startPrediction+l-1)[l-1])
        except ValueError:
            pred.append(tsValuesLog[startPrediction+i-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_'+str(p)+'_'+str(q)] = mae_
PredictionOutputRMSE['AR_'+str(p)+'_'+str(q)] = rmse_

Wall time: 54.3 s


In [12]:
PredictionOutputMAE

,RandomWalk,RandomWalkWithDrift,AR_1_1,AR_2_0
monthly,0.005169,0.006898,0.004725,0.004687
3_month,0.013316,0.018713,0.012316,0.012268
6_month,0.022940,0.033806,0.021303,0.021242
12_month,0.037548,0.060703,0.033415,0.033281


In [7]:
tsValuesLog_d_1 = [0]
tsValuesLog_d_1 += [tsValuesLog[i+1]-tsValuesLog[i] for i in range(totalData-1)]

In [14]:
mae_ = []
rmse_ = []
p=1
q=0
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        model = ARMA(tsValuesLog_d_1[i:i+startPrediction],order=(p,q))
        result = model.fit()
        pred.append(sum(result.predict(start=startPrediction,end=startPrediction+l-1))+tsValuesLog[i+startPrediction-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_Diff_'+str(p)+'_'+str(q)] = mae_
PredictionOutputRMSE['AR_Diff_'+str(p)+'_'+str(q)] = rmse_

In [15]:
%%time
mae_ = []
rmse_ = []
p=0
q=1
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        model = ARMA(tsValuesLog_d_1[i:i+startPrediction],order=(p,q))
        result = model.fit()
        pred.append(sum(result.predict(start=startPrediction,end=startPrediction+l-1))+tsValuesLog[i+startPrediction-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_Diff_'+str(p)+'_'+str(q)] = mae_
PredictionOutputRMSE['AR_Diff_'+str(p)+'_'+str(q)] = rmse_

C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Wall time: 27.7 s


In [27]:
#Unobserved model
import statsmodels.api as sm

class UnobservedComponentModel:
    def __init__(self):
        self.model_name = "UnobservedComponentModel"
        return

    def fit(self, ts):
        unrestricted_model = {
            'level': 'local linear trend', 'cycle': False , 'seasonal': 12
        }

        model = sm.tsa.UnobservedComponents(endog=ts, **unrestricted_model)
        self.trained_model = model.fit()
        return self

    def predict(self, next_n_prediction):
        prediction = self.trained_model.forecast(steps=next_n_prediction)
        return prediction

In [28]:
%%time
mae_ = []
rmse_ = []
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-(l-1)):
        model = UnobservedComponentModel()
        model.fit(tsValuesLog[i:i+startPrediction])
        pred.append(model.predict(l)[l-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))

C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check

Wall time: 10min 10s


In [25]:
PredictionOutputMAE["UnobservedComponentModel"] = mae_
PredictionOutputRMSE["UnobservedComponentModel"] = rmse_

In [29]:
PredictionOutputMAE["UnobservedComponentModel_With_Season"] = mae_
PredictionOutputRMSE["UnobservedComponentModel_With_Season"] = rmse_

In [30]:
PredictionOutputMAE

,RandomWalk,RandomWalkWithDrift,AR_1_1,AR_2_0,AR_Diff_1_0,AR_Diff_0_1,UnobservedComponentModel,UnobservedComponentModel_With_Season
monthly,0.005169,0.006898,0.004725,0.004687,0.006389,0.006443,0.005146,0.007007
3_month,0.013316,0.018713,0.012316,0.012268,0.018245,0.018308,0.013557,0.016231
6_month,0.022940,0.033806,0.021303,0.021242,0.033436,0.033531,0.023917,0.026154
12_month,0.037548,0.060703,0.033415,0.033281,0.059857,0.059959,0.039501,0.040267


In [31]:
PredictionOutputRMSE

,RandomWalk,RandomWalkWithDrift,AR_1_1,AR_2_0,AR_Diff_1_0,AR_Diff_0_1,UnobservedComponentModel,UnobservedComponentModel_With_Season
monthly,0.007606,0.008768,0.007134,0.007118,0.008284,0.008327,0.007576,0.009878
3_month,0.018230,0.022518,0.017257,0.017225,0.022024,0.022118,0.018885,0.022389
6_month,0.029523,0.039861,0.027466,0.027380,0.039353,0.039471,0.030886,0.034990
12_month,0.045839,0.071523,0.041231,0.041037,0.070519,0.070649,0.047310,0.048568


In [32]:
PredictionOutputMAE.to_csv("EXCHUS_MAE.csv")

In [33]:
PredictionOutputRMSE.to_csv("EXCHUS_RMAE.csv")